In [1]:
#This cell demonstrates a 2 Legged-Oauth call to Blackboard Learn
try:
    import Config
except ModuleNotFoundError:
    print("Config module not found. Did you copy Config_template.py to Config.py and set values?")
print("hello")
from bbrest import BbRest
bb = BbRest(Config.config['key'], Config.config['secret'], Config.config['url'])
bbexp = bb.expiration()
print(bbexp)
r = bb.GetUser(userId='mkauffman-student')
print (r.json())
print ("\n")
r = await bb.call('GetUser', userId='userName:mkauffman')
print(r.json())

hello
in 59 minutes
{'id': '_7_1', 'uuid': '4cdad5559fb84f44beeb889e0093b1c8', 'externalId': 'mkauffman-student', 'dataSourceId': '_2_1', 'userName': 'mkauffman-student', 'studentId': 'mkauffman-student', 'educationLevel': 'Unknown', 'gender': 'Unknown', 'created': '2024-01-09T16:31:34.745Z', 'modified': '2024-02-22T00:45:28.239Z', 'lastLogin': '2024-02-22T00:45:28.238Z', 'institutionRoleIds': ['STUDENT'], 'systemRoleIds': ['User'], 'availability': {'available': 'Yes'}, 'name': {'given': 'Mark (Student)', 'family': 'Kauffman', 'preferredDisplayName': 'GivenName'}, 'contact': {'email': 'markkauffman2000@gmail.com'}, 'avatar': {'viewUrl': 'https://kauffmantest390082.ddns.net/learn/api/public/v1/users/_7_1/avatar', 'source': 'Default'}}


{'id': '_6_1', 'uuid': '038db44f29144f548e1ce2163b533cb0', 'externalId': 'mkauffman', 'dataSourceId': '_2_1', 'userName': 'mkauffman', 'studentId': 'mkauffman', 'educationLevel': 'Unknown', 'gender': 'Unknown', 'created': '2024-01-04T19:00:05.777Z', 'mod

In [ ]:
# This cell demonstrates a Flask Application that makes 2 Legged-Oauth calls to Blackboard Learn.
# Notice how the Config and the import of BbRest to bb in the prior cell are available in this cell.
from flask import Flask

bb = BbRest(Config.config['key'], Config.config['secret'], Config.config['url'])
bbexp = bb.expiration()
print(bbexp)

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, world!'

@app.route('/users')
def users():
    r = bb.GetUsers()
    return(r.json())

if __name__ == '__main__':
    app.run()

in 58 minutes
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Feb/2024 16:14:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2024 16:14:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Feb/2024 16:14:59] "GET /users HTTP/1.1" 200 -
